In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Tablas

In [2]:
MafiActual = pd.read_csv('MafiActual.csv',
                         dtype=str)
MafiAnterior = pd.read_csv('MafiAnterior.csv',
                         dtype=str)
ColNum = {'origin_currency_balance_amount':np.float64,
          'mthly_lcur_average_bal_amount':np.float64,
          'current_bal_local_curncy_amount':np.float64,
          'mthly_lcur_average_bal1_amount':np.float64}
MafiActual, MafiAnterior = MafiActual.astype(ColNum), MafiAnterior.astype(ColNum)

# Atributos

In [3]:
AccActual = pd.read_csv('AccActual.csv',
                         dtype=str)
AttActual = pd.read_csv('AttActual.csv',
                         dtype=str)
AttActual = AttActual.rename(columns={'account_opening_date': 'opening_date',
                                      'original_expiry_date': 'last_expiry_date'})
CardActual = pd.read_csv('CardActual.csv',
                         dtype=str)
AtributosActual = pd.concat([AccActual,AttActual,CardActual])

AccAnterior = pd.read_csv('AccAnterior.csv',
                         dtype=str)
AttAnterior = pd.read_csv('AttAnterior.csv',
                         dtype=str)
AttAnterior = AttAnterior.rename(columns={'account_opening_date': 'opening_date',
                                      'original_expiry_date': 'last_expiry_date'})
CardAnterior = pd.read_csv('CardAnterior.csv',
                         dtype=str)
AtributosAnterior = pd.concat([AccAnterior,AttAnterior,CardAnterior])
Atributos = pd.concat([AtributosActual,AtributosAnterior])
Atributos['load_date'] = Atributos['load_date'].str[:7]

# Rorc

In [4]:
RorcActual = pd.read_csv('RorcActual.csv',
                         dtype=str)
RorcAnterior = pd.read_csv('RorcAnterior.csv',
                         dtype=str)
Rorc = pd.concat([RorcActual,RorcAnterior])
Rorc['cutoff_date'] = Rorc['cutoff_date'].str[:7]
Rorc = Rorc.rename(columns={'cutoff_date': 'load_date'})

# OppAtt

In [5]:
OpAttActual = pd.read_csv('OpAttActual.csv',
                         dtype=str)
OpAttAnterior = pd.read_csv('OpAttAnterior.csv',
                         dtype=str)
OpAtt = pd.concat([OpAttActual,OpAttAnterior])
OpAtt['cutoff_date'] = OpAtt['cutoff_date'].str[:7]
OpAtt = OpAtt.rename(columns={'cutoff_date': 'load_date'})

# Refina

In [6]:
RelatedActual = pd.read_csv('RelatedActual.csv',
                  dtype=str,
                  usecols=['source_contract_id','dest_contract_id','relation_type','information_date'])
RelatedAnterior = pd.read_csv('RelatedAnterior.csv',
                  dtype=str,
                  usecols=['source_contract_id','dest_contract_id','relation_type','information_date'])
RelatedGen = pd.concat([RelatedActual,RelatedAnterior])
RelatedGen = RelatedGen[RelatedGen['relation_type'].isin(['002010','002003'])]

# Productos

In [42]:
Productos = pd.read_csv('CatProdCom.csv',dtype=str,encoding='latin-1', usecols=['CD_PROD_COMER','CD_SUBPROD_COMER','PRODCOME'])
Productos.columns = ['commercial_product_id','commercial_subproduct_id','product_name']

# Catalogo Clientes

In [43]:
Clientes = pd.read_csv('CatClientes.csv',dtype=str,encoding='latin-1')
Clientes.columns = ['customer_id','group'] 

----

# Proceso

In [110]:
Mafi = MafiActual.merge(MafiAnterior, on = ['contract_id', 'customer_id', 'gl_account_id',
                                            'commercial_product_id', 'commercial_subproduct_id',
                                            'sender_application_id', 'gl_account_branch_id', 'currency_id'],
                                            how='outer',
                                            suffixes=('_actual', '_anterior'))
Mafi['load_date'] = Mafi['load_date_actual'].fillna(Mafi['load_date_anterior'])
Mafi = Mafi.drop(['load_date_actual','load_date_anterior'],axis=1)
cols = ['load_date'] + [col for col in Mafi.columns if col != 'load_date']
Mafi = Mafi[cols]
Mafi['load_date'] = Mafi['load_date'].str[:7]
Mafi['mthly_lcur_average_bal_amount_actual'] = Mafi['mthly_lcur_average_bal_amount_actual'].fillna(0)
Mafi['origin_currency_balance_amount_actual'] = Mafi['origin_currency_balance_amount_actual'].fillna(0)
Mafi['mthly_lcur_average_bal1_amount_actual'] = Mafi['mthly_lcur_average_bal1_amount_actual'].fillna(0)
Mafi['current_bal_local_curncy_amount_actual'] = Mafi['current_bal_local_curncy_amount_actual'].fillna(0)
Mafi['mthly_lcur_average_bal_amount_anterior'] = Mafi['mthly_lcur_average_bal_amount_anterior'].fillna(0)
Mafi['origin_currency_balance_amount_anterior'] = Mafi['origin_currency_balance_amount_anterior'].fillna(0)
Mafi['mthly_lcur_average_bal1_amount_anterior'] = Mafi['mthly_lcur_average_bal1_amount_anterior'].fillna(0)
Mafi['current_bal_local_curncy_amount_anterior'] = Mafi['current_bal_local_curncy_amount_anterior'].fillna(0)
Mafi['information_process'] = MafiActual['load_date']
Mafi.shape

(636092, 18)

In [111]:
Mafi['var_medio'] = Mafi['mthly_lcur_average_bal_amount_actual'] - Mafi['mthly_lcur_average_bal_amount_anterior']
Mafi['var_puntual'] = Mafi['origin_currency_balance_amount_actual'] - Mafi['origin_currency_balance_amount_anterior']
Mafi['var_medio_origen'] = Mafi['mthly_lcur_average_bal1_amount_actual'] - Mafi['mthly_lcur_average_bal1_amount_anterior']
Mafi['var_puntual_origen'] = Mafi['current_bal_local_curncy_amount_actual'] - Mafi['current_bal_local_curncy_amount_anterior']

In [114]:
Mafi1 = Mafi.merge(Atributos,
                   on = ['contract_id','load_date'],
                   how = 'left').merge(Rorc,
                                       on = ['contract_id','load_date'],
                                       how = 'left').merge(OpAtt,
                                                           left_on=['source_contract1_id','load_date'],
                                                           right_on=['main_operation_id','load_date'], how='left')
Mafi1.shape
ColumnasFecha = ['load_date','opening_date','last_expiry_date', 'contract_cancel_date',  'start_date', 'expiration_date','information_process']
Mafi1[ColumnasFecha] = Mafi1[ColumnasFecha].apply(lambda x: x.str[:7])
Mafi1[ColumnasFecha] = Mafi1[ColumnasFecha].apply(lambda x: pd.to_datetime(x, format='%Y-%m'))

In [115]:
# Campo Entrada
Mafi1['Entrada'] = np.where(
    (Mafi1['load_date'].dt.year == Mafi1['opening_date'].dt.year) &
    (Mafi1['load_date'].dt.month == Mafi1['opening_date'].dt.month),
    'alta',
    None)

# Campo Salida

Mafi1['Salida'] = np.where(
    (Mafi1['last_expiry_date'].dt.year == Mafi1['contract_cancel_date'].dt.year) & 
    (Mafi1['last_expiry_date'].dt.month == Mafi1['contract_cancel_date'].dt.month),
    'natural',
    np.where(
        (Mafi1['opening_date'].dt.year == Mafi1['contract_cancel_date'].dt.year) & 
        (Mafi1['opening_date'].dt.month == Mafi1['contract_cancel_date'].dt.month) &
        (Mafi1['opening_date'].dt.year == Mafi1['load_date'].dt.year) & 
        (Mafi1['opening_date'].dt.month == Mafi1['load_date'].dt.month),
        'alta-baja',
        np.where(
            ((Mafi1['contract_cancel_date'].dt.year == Mafi1['load_date'].dt.year) & 
            (Mafi1['contract_cancel_date'].dt.month == Mafi1['load_date'].dt.month))&
            (Mafi1['contract_cancel_date'].dt.year < Mafi1['last_expiry_date'].dt.year) |
            ((Mafi1['contract_cancel_date'].dt.year == Mafi1['last_expiry_date'].dt.year) &
             (Mafi1['contract_cancel_date'].dt.month < Mafi1['last_expiry_date'].dt.month)),
            'prepago',
            None
        )
    )
)


# Campo linea
Mafi1['linea'] = np.select(
        [
            Mafi1['source_contract1_id'].isnull() & Mafi1['source_contract_id'].notnull(),
            Mafi1['source_contract_id'].isnull() & Mafi1['source_contract1_id'].notnull(),
            Mafi1['source_contract_id'].notnull() & Mafi1['source_contract1_id'].notnull(),
            Mafi1['source_contract_id'].isnull() & Mafi1['source_contract1_id'].isnull()
        ],
        [
            'comprometida',
            'no comprometida',
            'comprometida',
            'no linea'
        ],
        default=None
    )

# Campo New Money
Mafi1['new_money'] = np.where(
    (Mafi1['start_date'].dt.year == Mafi1['opening_date'].dt.year) & 
    (Mafi1['start_date'].dt.month == Mafi1['opening_date'].dt.month)&
    (Mafi1['start_date'].dt.year == Mafi1['load_date'].dt.year) & 
    (Mafi1['start_date'].dt.month == Mafi1['load_date'].dt.month),
    '1',
    np.where(
        (Mafi1['linea'] == 'no linea') &
        ((Mafi1['opening_date'].dt.year == Mafi1['information_process'].dt.year) & 
        (Mafi1['opening_date'].dt.month == Mafi1['information_process'].dt.month)),
        '2',
        None
    )
)


In [116]:
Mafi1 = Mafi1.merge(Productos,
                   on = ['commercial_product_id','commercial_subproduct_id'],
                   how = 'left').merge(Clientes,
                                       on = 'customer_id',
                                       how = 'left')

In [119]:
pd.pivot_table(Mafi1,
               index = ['Salida'],
               values=['origin_currency_balance_amount_actual','origin_currency_balance_amount_anterior'],
               dropna=False,
               fill_value=0,
               margins=True,
               aggfunc='sum')/1000000

,origin_currency_balance_amount_actual,origin_currency_balance_amount_anterior
Salida,,
alta-baja,0.00,0.00
natural,0.00,16873.42
prepago,0.00,36598.20
All,272878.63,275046.08


In [118]:
pd.pivot_table(Mafi1,
               index = ['new_money'],
               values=['origin_currency_balance_amount_actual','origin_currency_balance_amount_anterior'],
               dropna=False,
               fill_value=0,
               margins=True,
               aggfunc='sum')/1000000

,origin_currency_balance_amount_actual,origin_currency_balance_amount_anterior
new_money,,
1,2614.30,0.00
2,2064.77,0.00
All,272878.63,275046.08
